In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
import json
import os

In [ ]:
mask = np.zeros([img.shape[0], img.shape[1], 1], dtype=np.uint8)

for contour in contours:
    contour = np.array(contour)
    xpoints = contour.transpose()[0][0]
    ypoints = contour.transpose()[1][0]
    
    rr, cc = skimage.draw.polygon(xpoints, ypoints)
    
    mask[cc, rr, 0] = img[cc, rr, 0]
    
cv.imshow('asdf', mask)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
img = cv2.imread("datasets/test_mask/mask_Object1_3.png")
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

ret, thresh = cv2.threshold(imgray, 127, 255, 0)

contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

all_xpoints = []
all_ypoints = []

for contour in contours:
    xpoints = contour.transpose()[0][0]
    ypoints = contour.transpose()[1][0]
    
    all_xpoints.append(xpoints)
    all_ypoints.append(ypoints)

# cv2.imshow('test', img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [10]:
def find_contours(img):
    # Grayscale image 
    imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Threshold the image
    ret, thresh = cv2.threshold(imgray, 127, 255, 0)

    # Find the contours
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Return an empty array if it did not find any contours
    if len(contours) == 0:
        return [], []
    
    all_xpoints = []
    all_ypoints = []

    for contour in contours:
        xpoints = contour.transpose()[0][0]
        ypoints = contour.transpose()[1][0]

        all_xpoints.append(xpoints.tolist())
        all_ypoints.append(ypoints.tolist())

    return all_xpoints, all_ypoints

In [11]:
class Mask(object):
    def __init__(self, file):
        self.name = file[0].split('_Object')[1].split('.')[0]
        self.classification = self.name.split('_')[0]
        self.contours = find_contours(cv2.imread(file[0]))


class Image(object):
    def __init__(self, file):
        print(file)
        self.image = cv2.imread(file)
        self.name = file.split('/')[1]
        self.mask = self.get_mask()

    def get_mask(self):
        path = "./mask/mask_Object" + self.name.split('_Object')[1]
        return Mask(glob.glob(path))

In [12]:
# Save the points to a json file
images = []

for file in glob.glob("val_images/*.png"):
    images.append(Image(file))
    
data = {}

for image in images:
    region = {}
    
    region['name'] = image.mask.name
    region['class'] = image.mask.classification

    x_points, y_points = image.mask.contours
    
    region['all_points_x'] = x_points
    region['all_points_y'] = y_points
    
    image_information = {}
    height, width, _ = image.image.shape
    image_information['filename'] = image.name
    image_information['width'] = width
    image_information['height'] = height
    image_information['region'] = region
    
    data['image_' + image.name] = image_information
    
with open('val_images/annotations.json', 'w') as json_file:
    json.dump(data, json_file)

val_images/image_Object1_1747.png
val_images/image_Object1_1753.png
val_images/image_Object3_1099.png
val_images/image_Object3_1072.png
val_images/image_Object1_1960.png
val_images/image_Object1_1974.png
val_images/image_Object2_819.png
val_images/image_Object3_1066.png
val_images/image_Object2_831.png
val_images/image_Object1_1784.png
val_images/image_Object1_1790.png
val_images/image_Object1_1948.png
val_images/image_Object2_825.png
val_images/image_Object4_1942.png
val_images/image_Object4_1956.png
val_images/image_Object4_1981.png
val_images/image_Object4_1995.png
val_images/image_Object5_1017.png
val_images/image_Object5_1003.png
val_images/image_Object4_1836.png
val_images/image_Object5_1188.png
val_images/image_Object4_1822.png
val_images/image_Object3_997.png
val_images/image_Object3_983.png
val_images/image_Object3_968.png
val_images/image_Object5_1163.png
val_images/image_Object3_954.png
val_images/image_Object5_1177.png
val_images/image_Object3_940.png
val_images/image_Objec

val_images/image_Object3_979.png
val_images/image_Object2_767.png
val_images/image_Object2_773.png
val_images/image_Object2_798.png
val_images/image_Object1_1839.png
val_images/image_Object1_1811.png
val_images/image_Object1_1805.png
val_images/image_Object1_1804.png
val_images/image_Object1_1810.png
val_images/image_Object1_1838.png
val_images/image_Object2_799.png
val_images/image_Object2_772.png
val_images/image_Object2_766.png
val_images/image_Object3_978.png
val_images/image_Object3_944.png
val_images/image_Object5_1173.png
val_images/image_Object3_950.png
val_images/image_Object5_1167.png
val_images/image_Object4_1826.png
val_images/image_Object5_1198.png
val_images/image_Object4_1832.png
val_images/image_Object3_987.png
val_images/image_Object3_993.png
val_images/image_Object4_1991.png
val_images/image_Object4_1985.png
val_images/image_Object5_1007.png
val_images/image_Object5_1013.png
val_images/image_Object4_1952.png
val_images/image_Object4_1946.png
val_images/image_Object3_1

val_images/image_Object4_1844.png
val_images/image_Object5_1105.png
val_images/image_Object3_932.png
val_images/image_Object5_1111.png
val_images/image_Object3_926.png
val_images/image_Object4_1893.png
val_images/image_Object5_1139.png
val_images/image_Object4_1887.png
val_images/image_Object2_738.png
val_images/image_Object1_1899.png
val_images/image_Object2_704.png
val_images/image_Object2_710.png
val_images/image_Object1_1872.png
val_images/image_Object1_1866.png
val_images/image_Object1_1867.png
val_images/image_Object1_1873.png
val_images/image_Object2_711.png
val_images/image_Object2_705.png
val_images/image_Object1_1898.png
val_images/image_Object2_739.png
val_images/image_Object4_1886.png
val_images/image_Object5_1138.png
val_images/image_Object4_1892.png
val_images/image_Object3_927.png
val_images/image_Object5_1110.png
val_images/image_Object3_933.png
val_images/image_Object5_1104.png
val_images/image_Object4_1845.png
val_images/image_Object4_1851.png
val_images/image_Object4

val_images/image_Object3_1009.png
val_images/image_Object2_862.png
val_images/image_Object3_1035.png
val_images/image_Object1_1927.png
val_images/image_Object1_1933.png
val_images/image_Object3_1021.png
val_images/image_Object5_1093.png
val_images/image_Object4_1939.png
val_images/image_Object5_1087.png
val_images/image_Object4_1905.png
val_images/image_Object4_1911.png
val_images/image_Object5_1050.png
val_images/image_Object5_1044.png
val_images/image_Object5_1078.png
val_images/image_Object4_1859.png
val_images/image_Object4_1871.png
val_images/image_Object4_1865.png
val_images/image_Object5_1124.png
val_images/image_Object3_913.png
val_images/image_Object5_1130.png
val_images/image_Object3_907.png
val_images/image_Object5_1118.png
val_images/image_Object2_719.png
val_images/image_Object2_725.png
val_images/image_Object1_1890.png
val_images/image_Object1_1884.png
val_images/image_Object2_731.png
val_images/image_Object1_1853.png
val_images/image_Object1_1847.png
val_images/image_Obj

val_images/image_Object2_792.png
val_images/image_Object2_786.png
val_images/image_Object1_1833.png
val_images/image_Object2_787.png
val_images/image_Object1_1832.png
val_images/image_Object1_1826.png
val_images/image_Object2_793.png
val_images/image_Object2_744.png
val_images/image_Object2_750.png
val_images/image_Object2_778.png
val_images/image_Object5_1179.png
val_images/image_Object3_972.png
val_images/image_Object5_1145.png
val_images/image_Object3_966.png
val_images/image_Object5_1151.png
val_images/image_Object4_1810.png
val_images/image_Object3_999.png
val_images/image_Object4_1804.png
val_images/image_Object5_1186.png
val_images/image_Object4_1838.png
val_images/image_Object5_1192.png
val_images/image_Object5_1019.png
val_images/image_Object5_1031.png
val_images/image_Object5_1025.png
val_images/image_Object4_1964.png
val_images/image_Object4_1970.png
val_images/image_Object4_1958.png
val_images/image_Object3_1054.png
val_images/image_Object1_1946.png
val_images/image_Object1

In [ ]:
#For testing
import random

annotations = json.load(open(os.path.join("./val_images/annotations.json")))
annotations = list(annotations.values())

annotation = annotations[0]

fin_img = cv2.imread("./val_images/" + annotation['filename'])

x_points = annotation['region']['all_points_x']
y_points = annotation['region']['all_points_y']
# print(len(y_points) == len(x_points))

color = random.choice([[0,0,255], [255, 0, 0], [0, 255, 0], [255, 255, 0], [0, 255, 255], [255, 0, 255]])

points = np.array([x_points, y_points]).transpose()

cv2.polylines(fin_img, [points], True, color, 2)

# Show
cv2.imshow(annotation['filename'], fin_img)

cv2.waitKey(0)
cv2.destroyAllWindows()